In [145]:
# ********** Import Statements ************************************************************ #
import sys
import spotipy
import spotipy.util as util

import pandas as pd
import math



In [146]:
user_ids = {'Cassie':'1242091675',
            'Caity':'12166088827'}

In [147]:
# Spotify credentials
scope = 'user-library-read'
#client_id removed
#client_secret removed
redirect_uri = 'http://localhost:1410/'

In [148]:
def connect_spotify(user_id):
    token = util.prompt_for_user_token(user_id,
                                       scope,
                                       client_id,
                                       client_secret,
                                       redirect_uri)

    sp = spotipy.Spotify(auth=token)
    return sp

In [149]:
sp = connect_spotify(user_ids['Cassie'])

In [32]:
def get_all_playlists(sp, user_id):
    total = sp.user_playlists(user_id)['total']
    limit = 50

    playlists = []
    for x in range(0, total, limit):
        new_playlist = sp.user_playlists(user_id, limit=limit, offset=x)
        playlists += new_playlist['items']

    playlist_df = pd.DataFrame(playlists)
    playlist_df['owner'] =  playlist_df['owner'].apply(lambda x: x['id'])

    # prep for return
    playlist_df = playlist_df[['collaborative', 'id', 'name', 'owner', 'public']]
    playlist_df = playlist_df.rename(columns={'id':'playlist_id', 
                                              'owner':'owner_id',
                                              'name':'playlist_name'})

    #playlist_tracks_df = playlist_df['playlist_id'].apply()
    return playlist_df

In [50]:
playlists = get_all_playlists(sp, user_ids['Cassie'])

In [52]:
p = sp.user_playlist(user_ids['Cassie'], playlists['playlist_id'][0])
p.keys()

dict_keys(['collaborative', 'description', 'external_urls', 'followers', 'href', 'id', 'images', 'name', 'owner', 'primary_color', 'public', 'snapshot_id', 'tracks', 'type', 'uri'])

In [57]:
playlist_id = p['id']

In [79]:
p['tracks']['items'][0]['track']['artists']#.keys()

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/08VxLl1rmtFqsvoAmCVaNH'},
  'href': 'https://api.spotify.com/v1/artists/08VxLl1rmtFqsvoAmCVaNH',
  'id': '08VxLl1rmtFqsvoAmCVaNH',
  'name': 'The Bengsons',
  'type': 'artist',
  'uri': 'spotify:artist:08VxLl1rmtFqsvoAmCVaNH'}]

In [76]:
p['tracks']['items'][0]['track']['id']

'39YGM5ee7zW23FM7gmTpMk'

In [75]:
p['tracks']['items'][0]['track']['name']

'Hundred Days'

In [ ]:
if playlist['owner']['id'] == usr:
            results = sp.user_playlist(usr, playlist['id'],
                fields="tracks,next")
            tracks = results['tracks']
            for i, item in enumerate(tracks['items']):
                track = item['track']

                track_deets = pd.DataFrame({'playlist_id': playlist['id'],
                                           'playlist': playlist['name'], 
                                           'artist': track['artists'][0]['name'], 
                                           'title': track['name'],
                                           'song_id': track['id']})
                song_deets = pd.DataFrame(sp.audio_features(track['id']))
                playlist_deets = pd.merge(track_deets, song_deets, 
                                          how='left', 
                                          left_on = 'song_id', 
                                          right_on='id')
                temp_df = temp_df.append(playlist_deets)

In [160]:
def extract_values(obj, key):
    """Pull all values of specified key from nested JSON."""
    arr = []

    def extract(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    results = extract(obj, arr, key)
    return results

In [161]:
playlist = sp.user_playlists(user_ids['Cassie'])

In [170]:
extract_values(playlist['items'], 'name')

["October '19",
 'Cleaning',
 'Perfect Harmonies',
 'Discover Weekly',
 'Release Radar',
 'Alarm songs',
 '0',
 '1',
 'Morning Playlist',
 "September '19",
 "August '19",
 "July '19",
 "June '19",
 "May '19",
 "April '19",
 "March '19",
 "February '19",
 "January '19",
 "December '18",
 "November '18",
 "October '18",
 "September '18",
 "August '18",
 "July '18",
 'SoundHound',
 'Release Radar Archive',
 'Discover Weekly Archive',
 'Inspired By Warriors',
 'Breathe me in.',
 'Inside my head.',
 'Mind on Repeat',
 'Past Obsessions',
 'Teenage Cassie',
 'Summer 2017',
 'Hamilton',
 'Beats to think to',
 'Sleep',
 '**DEEP FOCUS** Orchestral Post-Minimalism',
 'Deep Focus',
 'Cinematic Chillout',
 'Cinematic Classical',
 'Soundwave CD',
 'האהובים עלי',
 'Morning Acoustic',
 'Chill Lah!',
 'PTX Christmas Songs',
 'Acapella Christmas',
 'Happy Beats',
 "It's ALT Good!",
 'Mood Booster']

In [159]:
df

,0
href,https://api.spotify.com/v1/users/1242091675/pl...
items_0_collaborative,False
items_0_external_urls_spotify,https://open.spotify.com/playlist/1Pp4hppaHlPr...
items_0_href,https://api.spotify.com/v1/playlists/1Pp4hppaH...
items_0_id,1Pp4hppaHlPrSYUbXTHMPk
items_0_images_0_height,640
items_0_images_0_url,https://i.scdn.co/image/ab67616d0000b273dc2d1e...
items_0_images_0_width,640
items_0_name,October '19
items_0_owner_display_name,Cassie Beth


In [153]:
json_normalize(sp.user_playlists(user_ids['Cassie'])).T

,0
href,https://api.spotify.com/v1/users/1242091675/pl...
items,"[{'collaborative': False, 'external_urls': {'s..."
limit,50
next,https://api.spotify.com/v1/users/1242091675/pl...
offset,0
previous,None
total,154


In [27]:
for i, item in enumerate(sp.user_playlist(user_ids['Cassie'],'1Pp4hppaHlPrSYUbXTHMPk')['tracks']['items']):
    print(i, item)

0 {'added_at': '2019-10-07T02:22:13Z', 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/1242091675'}, 'href': 'https://api.spotify.com/v1/users/1242091675', 'id': '1242091675', 'type': 'user', 'uri': 'spotify:user:1242091675'}, 'is_local': False, 'primary_color': None, 'track': {'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/08VxLl1rmtFqsvoAmCVaNH'}, 'href': 'https://api.spotify.com/v1/artists/08VxLl1rmtFqsvoAmCVaNH', 'id': '08VxLl1rmtFqsvoAmCVaNH', 'name': 'The Bengsons', 'type': 'artist', 'uri': 'spotify:artist:08VxLl1rmtFqsvoAmCVaNH'}], 'available_markets': ['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DE', 'DK', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FR', 'GB', 'GR', 'GT', 'HK', 'HN', 'HU', 'ID', 'IE', 'IL', 'IS', 'IT', 'JO', 'JP', 'KW', 'LB', 'LI', 'LT', 'LU', 'LV', 'MA', 'MC', 'MT', 'MX', 'MY', 'NI', 'NL', 'NO', 'NZ', 'OM', 'PA', 'PE',

In [21]:
playlists.

,collaborative,playlist_id,playlist_name,owner_id,public
0,False,1Pp4hppaHlPrSYUbXTHMPk,October '19,1242091675,False
1,False,5Vv7ob001nbg7rPf31idQE,Cleaning,1242091675,True
2,False,1Sc1UfKCURYymoMQ6WeScn,Perfect Harmonies,1242091675,True
3,False,1rmBxO0m8o0VV4YWvcsBIA,Oktober '19,1242091675,False
4,False,37i9dQZEVXcXaEd1YMLO1z,Discover Weekly,spotify,False
5,False,37i9dQZEVXbxfea14yrYzN,Release Radar,spotify,False
6,False,5lahvR0jvnaeRgX7Kw56SP,Alarm songs,1242091675,True
7,False,2KDvDoJQMjldDWaCBpxrgu,0,1242091675,True
8,False,7BdKV40GJbm9RT3ILusyee,1,1242091675,True
9,False,3sPSfyVUzE54fqqKmr39hA,Morning Playlist,1242091675,False


In [90]:
sp.user_playlists(user_ids['Cassie'])['items'][0]['id']

{'collaborative': False,
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/1Pp4hppaHlPrSYUbXTHMPk'},
 'href': 'https://api.spotify.com/v1/playlists/1Pp4hppaHlPrSYUbXTHMPk',
 'id': '1Pp4hppaHlPrSYUbXTHMPk',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab67616d0000b273dc2d1e5f0197715796f1a759',
   'width': 640}],
 'name': "October '19",
 'owner': {'display_name': 'Cassie Beth',
  'external_urls': {'spotify': 'https://open.spotify.com/user/1242091675'},
  'href': 'https://api.spotify.com/v1/users/1242091675',
  'id': '1242091675',
  'type': 'user',
  'uri': 'spotify:user:1242091675'},
 'primary_color': None,
 'public': False,
 'snapshot_id': 'MyxjNTNiNjEyNzViMzEwOWUwNDlhNDExZDQ0MWNjMDc1MjJjZWZjZDY5',
 'tracks': {'href': 'https://api.spotify.com/v1/playlists/1Pp4hppaHlPrSYUbXTHMPk/tracks',
  'total': 2},
 'type': 'playlist',
 'uri': 'spotify:playlist:1Pp4hppaHlPrSYUbXTHMPk'}

In [116]:
playlist_id = sp.user_playlists(user_ids['Cassie'])['items'][0]['id']

In [119]:
playlist_name = sp.user_playlists(user_ids['Cassie'])['items'][0]['name']

In [120]:
playlist_owner_id = sp.user_playlists(user_ids['Cassie'])['items'][0]['owner']['id']

In [122]:
one_playlist = sp.user_playlist(user_ids['Cassie'], playlist_id)

In [123]:
one_track = one_playlist['tracks']['items'][0]['track']#.keys()

In [131]:
temp = {'playlist_id': playlist_id,
 'playlist_name': playlist_name,
 'playlist_owner_id' : playlist_owner_id,
 'track_name':one_track['name'],
 'track_id':one_track['id'],
 'track_artist_id': one_track['artists'][0]['id'],
 'track_artist_name': one_track['artists'][0]['name']}

In [139]:
temp2 = {'playlist_id': playlist_id,
 'playlist_name': playlist_name,
 'playlist_owner_id' : playlist_owner_id,
 'track_name':one_track['name'],
 'track_id':one_track['id'],
 'track_artist_id': one_track['artists'][0]['id'],
 'track_artist_name': one_track['artists'][0]['name']}

In [358]:
recursive_skim(one_playlist['tracks']['items'][0]['track'])

0  * album <class 'dict'> 
1 	 * album_type <class 'str'> 
1 	 * artists <class 'list'> [0] <class 'dict'>
2 		 * external_urls <class 'dict'> 
3 			 * spotify <class 'str'> 
2 		 * href <class 'str'> 
2 		 * id <class 'str'> 
2 		 * name <class 'str'> 
2 		 * type <class 'str'> 
2 		 * uri <class 'str'> 
1 	 * available_markets <class 'list'> [0] <class 'str'>
1 	 * external_urls <class 'dict'> 
2 		 * spotify <class 'str'> 
1 	 * href <class 'str'> 
1 	 * id <class 'str'> 
1 	 * images <class 'list'> [0] <class 'dict'>
2 		 * height <class 'int'> 
2 		 * url <class 'str'> 
2 		 * width <class 'int'> 
1 	 * name <class 'str'> 
1 	 * release_date <class 'str'> 
1 	 * release_date_precision <class 'str'> 
1 	 * total_tracks <class 'int'> 
1 	 * type <class 'str'> 
1 	 * uri <class 'str'> 
0  * artists <class 'list'> [0] <class 'dict'>
1 	 * external_urls <class 'dict'> 
2 		 * spotify <class 'str'> 
1 	 * href <class 'str'> 
1 	 * id <class 'str'> 
1 	 * name <class 'str'> 
1 	 * type <

In [260]:
recursive_skim(playlist)

 * href <class 'str'> 
 * items <class 'list'> [0]
	 * collaborative <class 'bool'> 
	 * external_urls <class 'dict'> 
		 * spotify <class 'str'> 
	 * href <class 'str'> 
	 * id <class 'str'> 
	 * images <class 'list'> [0]
		 * height <class 'int'> 
		 * url <class 'str'> 
		 * width <class 'int'> 
	 * name <class 'str'> 
	 * owner <class 'dict'> 
		 * display_name <class 'str'> 
		 * external_urls <class 'dict'> 
			 * spotify <class 'str'> 
		 * href <class 'str'> 
		 * id <class 'str'> 
		 * type <class 'str'> 
		 * uri <class 'str'> 
	 * primary_color <class 'NoneType'> 
	 * public <class 'bool'> 
	 * snapshot_id <class 'str'> 
	 * tracks <class 'dict'> 
		 * href <class 'str'> 
		 * total <class 'int'> 
	 * type <class 'str'> 
	 * uri <class 'str'> 
 * limit <class 'int'> 
 * next <class 'str'> 
 * offset <class 'int'> 
 * previous <class 'NoneType'> 
 * total <class 'int'> 


In [235]:
for x in playlist.keys():
    level_0 = playlist[x]
    print(x, type(level_0))
    if type(level_0) in (list, dict):
        level_1 = level_0[0] if type(level_0) == list else level_0
        #print('\t* elements of', type(level_1))
        for y in level_1.keys():
            level_2 = level_1[y]
            print('\t*', y, type(level_2))
            if type(level_2) in (list, dict):
                level_3 = level_2[0] if type(level_2) == list else level_2
                #print('\t\t* elements of', type(level_3))
                for z in level_3.keys():
                    level_4 = level_3[z]
                    print('\t\t*', z, type(level_3))
                    if type(level_4) in (list, dict):
                        level_5 = level_4[0] if type(level_4) == list else level_4
            #if type(playlist[]) in (list, dict):
#         print(type(x))
#     for y in playlist[x].keys():
#         print('...', y)

href <class 'str'>
items <class 'list'>
	* collaborative <class 'bool'>
	* external_urls <class 'dict'>
		* spotify <class 'dict'>
	* href <class 'str'>
	* id <class 'str'>
	* images <class 'list'>
		* height <class 'dict'>
		* url <class 'dict'>
		* width <class 'dict'>
	* name <class 'str'>
	* owner <class 'dict'>
		* display_name <class 'dict'>
		* external_urls <class 'dict'>
		* href <class 'dict'>
		* id <class 'dict'>
		* type <class 'dict'>
		* uri <class 'dict'>
	* primary_color <class 'NoneType'>
	* public <class 'bool'>
	* snapshot_id <class 'str'>
	* tracks <class 'dict'>
		* href <class 'dict'>
		* total <class 'dict'>
	* type <class 'str'>
	* uri <class 'str'>
limit <class 'int'>
next <class 'str'>
offset <class 'int'>
previous <class 'NoneType'>
total <class 'int'>


### Figuring out parsing a single playlist

In [ ]:
playlist = Cassie.sp.user_playlist(Cassie.id, '1Pp4hppaHlPrSYUbXTHMPk')
track = playlist['tracks']['items'][0]
x = track['track']

data = {'playlist_id'  : playlist['id'],
        'playlist_name': playlist['name'],
        'added_by'     : track['added_by']['id'],
        'track_name' : x['name'],
        'track_id': x['id'],
        'track_artists': [artist['name'] for artist in x['artists']]
       }